In [401]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

import holidays
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

from datetime import datetime, timedelta
import utils


In [402]:
# df = df.rename(columns={
#     "id_compteur": "counter_id",
#     "nom_compteur": "counter_name",
#     "id": "site_id",
#     "name": "site_name",
#     "date": "date",
#     "installation_date": "counter_installation_date",
#     "coordinates": "coordinates",
#     "counter": "counter_technical_id"
# })

# columns_to_keep = [
#     "counter_id", "counter_name", "site_id", "site_name", "date",
#     "counter_installation_date", "coordinates", "counter_technical_id"
# ]
# df = df[columns_to_keep]

In [403]:
# # Drop rows where the 'coordinates' column is NaN
# df = df.dropna(subset=['coordinates'])

# # Reset index after dropping rows
# df = df.reset_index(drop=True)

# # Display the updated DataFrame
# df.head(2)

In [404]:
# import struct

# # Function to decode latitude and longitude from byte string
# def decode_coordinates(byte_string):
#     try:
#         header_size = 5  # Skip the first 5 bytes as a header
#         lat, lon = struct.unpack("dd", byte_string[header_size:])
#         return lat, lon
#     except Exception as e:
#         print(f"Error decoding: {e}")
#         return None, None

# # Apply the function to the DataFrame
# df["latitude"], df["longitude"] = zip(*df["coordinates"].apply(decode_coordinates))

# # Drop the original 'coordinates' column if no longer needed
# df = df.drop(columns=["coordinates"])

# # Display the updated DataFrame
# df.head(2)


In [405]:
# # Ensure the 'date' column is in datetime format
# df['date'] = pd.to_datetime(df['date'])

# # Format the 'date' column to 'YYYY-MM-DD HH:MM:SS'
# df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# # Display the updated DataFrame
# df.head(2)

In [406]:
# # Ensure the 'date' column is in datetime format
# df['date'] = pd.to_datetime(df['date'])

# # Find the earliest and latest dates
# earliest_date = df['date'].min()
# latest_date = df['date'].max()

# print(f"Earliest date: {earliest_date}")
# print(f"Latest date: {latest_date}")


In [407]:
df1 = pd.read_csv('data/2019_comptage-velo-donnees-compteurs-2.csv', delimiter=';')
df2 = pd.read_csv('data/comptage-velo-donnees-compteurs.csv', delimiter=';')
df3 = pd.read_csv('data/comptage-velo-donnees-compteurs (1).csv', delimiter=';')
df4 = pd.read_csv('data/comptage-velo-donnees-compteurs (2).csv', delimiter=';')


/var/folders/q_/t0vvj4pn7l9bcb6_mjhmgg080000gn/T/ipykernel_4201/114106817.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('data/comptage-velo-donnees-compteurs (1).csv', delimiter=';')


In [408]:
A, b = utils.get_train_data()
A.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
400125,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233
408305,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233
87516,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020
98518,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020
875137,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,"48.88529,2.32666",Y2H20073268,48.885290,2.326660


In [409]:
A["result"] = b

In [410]:
df = pd.concat([df2, df3], ignore_index=True)

In [411]:
df.tail()

,Identifiant du compteur,Nom du compteur,Identifiant du site de comptage,Nom du site de comptage,Comptage horaire,Date et heure de comptage,Date d'installation du site de comptage,Lien vers photo du site de comptage,Coordonnées géographiques
2247254,353277235-100063175,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,15.0,2021-12-31T19:00:00,21-07-21,https://filer.eco-counter-tools.com/file/9c/fd...,"48.88529,2.32666"
2247255,353277235-100063175,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,7.0,2021-12-31T20:00:00,21-07-21,https://filer.eco-counter-tools.com/file/9c/fd...,"48.88529,2.32666"
2247256,353277235-100063175,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,0.0,2021-12-31T21:00:00,21-07-21,https://filer.eco-counter-tools.com/file/9c/fd...,"48.88529,2.32666"
2247257,353277235-100063175,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,1.0,2021-12-31T22:00:00,21-07-21,https://filer.eco-counter-tools.com/file/9c/fd...,"48.88529,2.32666"
2247258,353277235-100063175,20 Avenue de Clichy SE-NO,100063175,20 Avenue de Clichy,0.0,2021-12-31T23:00:00,21-07-21,https://filer.eco-counter-tools.com/file/9c/fd...,"48.88529,2.32666"


In [412]:
# Rename columns to match the desired format
df = df.rename(columns={
    "Identifiant du compteur": "counter_id",
    "Nom du compteur": "counter_name",
    "Identifiant du site de comptage": "site_id",
    "Nom du site de comptage": "site_name",
    "Date et heure de comptage": "date",
    "Date d'installation du site de comptage": "counter_installation_date",
    "Coordonnées géographiques": "coordinates",
    "Comptage horaire": "result"
})


# Split 'coordinates' into 'latitude' and 'longitude'
def extract_lat_long(coords):
    if isinstance(coords, str) and ',' in coords:
        lat, long = coords.split(",")
        return float(lat), float(long)
    return None, None

df["latitude"], df["longitude"] = zip(*df["coordinates"].apply(extract_lat_long))

# Drop the 'coordinates' column as it is no longer needed
# combined_df = combined_df.drop(columns=["coordinates"])

# Rearrange columns to match the desired format
df = df[[
    "counter_id", "counter_name", "site_id", "site_name", "date",
    "counter_installation_date", "latitude", "longitude", "result"
]]

df.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,latitude,longitude,result
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01T01:00:00+01:00,2012-02-22,48.83511,2.33338,12.0
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01T03:00:00+01:00,2012-02-22,48.83511,2.33338,11.0
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01T00:00:00+01:00,2012-02-22,48.83511,2.33338,6.0
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01T02:00:00+01:00,2012-02-22,48.83511,2.33338,9.0
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01T05:00:00+01:00,2012-02-22,48.83511,2.33338,6.0


In [413]:
df['date'] = df['date'].astype(str).apply(lambda x : x.split('+')[0])
df['date'] = df['date'].replace('+', ' ')
df['date'] = pd.to_datetime(df['date'])

In [414]:
df.isna().sum()

counter_id                        0
counter_name                      0
site_id                           0
site_name                         0
date                              0
counter_installation_date    376680
latitude                          0
longitude                         0
result                            0
dtype: int64

In [415]:
# Perform an inner merge on 'latitude', 'longitude', and 'date'
matching_rows = pd.merge(A, df, on=['latitude', 'longitude', 'date', 'counter_name'])

# Count the number of matching rows
num_matching_rows = len(matching_rows)

# Display the result
print(f"Number of rows where 'latitude', 'longitude', and 'date' are the same: {num_matching_rows}")


Number of rows where 'latitude', 'longitude', and 'date' are the same: 740672


In [416]:
A.drop(columns=['coordinates', 'counter_installation_date', 'counter_technical_id'], inplace=True)
A.head(2)

,counter_id,counter_name,site_id,site_name,date,latitude,longitude,result
400125,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,48.840801,2.333233,1.609438
408305,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,48.840801,2.333233,1.386294


In [417]:
df.drop(columns=['counter_installation_date'], inplace=True)
df['result'] = np.log1p(df['result'].values)
df.head(2)

,counter_id,counter_name,site_id,site_name,date,latitude,longitude,result
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01 01:00:00,48.83511,2.33338,2.564949
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2020-01-01 03:00:00,48.83511,2.33338,2.484907


In [418]:
A.nunique()

counter_id        56
counter_name      56
site_id           30
site_name         30
date            8974
latitude          30
longitude         30
result           998
dtype: int64

In [419]:
df.nunique()

counter_id        224
counter_name      105
site_id            75
site_name          77
date            17543
latitude           84
longitude          85
result           1066
dtype: int64

In [420]:
# pip install unidecode
# !pip install rapidfuzz
import re 
from unidecode import unidecode
from rapidfuzz import fuzz, process

In [421]:
def standardize_string(s):
    s = re.sub(r'\(.*?\)', '', s)  # Remove text inside parentheses, including parentheses
    s = s.lower()  # Convert to lowercase
    s = unidecode(s)  # Remove accents
    # s = re.sub(r'\b(\w+)\b(?: \1\b)+', r'\1', s)  # Remove repeated words/phrases
    return s

df['site_name'] = df['site_name'].apply(standardize_string)

def map_similar_names(column, threshold=90):
    unique_values = df[column].unique()
    mapping = {}
    for value in unique_values:
        # Find the best match from already mapped values
        best_match = process.extractOne(value, mapping.keys(), scorer=fuzz.ratio)
        if best_match and best_match[1] > threshold:
            mapping[value] = mapping[best_match[0]]  # Map to existing group
        else:
            mapping[value] = value  # Keep as-is if no match found
    return mapping

site_name_mapping = map_similar_names('site_name')

df['site_name'] = df['site_name'].map(site_name_mapping)


In [422]:


def standardize_string(s):
    s = re.sub(r'\(.*?\)', '', s)  # Remove text inside parentheses, including parentheses
    s = s.lower()  # Convert to lowercase
    s = unidecode(s)  # Remove accents
    # s = re.sub(r'\b(\w+)\b(?: \1\b)+', r'\1', s)  # Remove repeated words/phrases
    return s

df['counter_name'] = df['counter_name'].apply(standardize_string)

def map_similar_names(column, threshold=90):
    unique_values = df[column].unique()
    mapping = {}
    for value in unique_values:
        # Find the best match from already mapped values
        best_match = process.extractOne(value, mapping.keys(), scorer=fuzz.ratio)
        if best_match and best_match[1] > threshold:
            mapping[value] = mapping[best_match[0]]  # Map to existing group
        else:
            mapping[value] = value  # Keep as-is if no match found
    return mapping

counter_name_mapping = map_similar_names('counter_name')

df['counter_name'] = df['counter_name'].map(counter_name_mapping)


In [423]:
# Define a function to replace IDs within each counter_name group
def replace_ids_in_group(group):
    # Iterate over the group rows to find the replacement ID
    for index, row in group.iterrows():
        parts = row['counter_id'].split('-')
        if len(parts) == 2 and parts[0] == str(row['site_id']) and parts[0] != parts[1]:
            # Found a valid replacement ID, update the group
            replacement_id = row['counter_id']
            group['counter_id'] = replacement_id  # Replace all with this ID
            break
    return group

# Apply the function to update the DataFrame
df = df.groupby('counter_name').apply(replace_ids_in_group)

# Optional: Reset the index if groupby creates a multi-index
df.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
df.nunique()


/var/folders/q_/t0vvj4pn7l9bcb6_mjhmgg080000gn/T/ipykernel_4201/1671627000.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('counter_name').apply(replace_ids_in_group)


counter_id         83
counter_name       83
site_id            75
site_name          70
date            17543
latitude           84
longitude          85
result           1066
dtype: int64

In [424]:
# Convert site_id in both DataFrames to integers (if not already)
df['site_id'] = df['site_id'].astype(int)
A['site_id'] = A['site_id'].astype(int)

# Create a mapping for site_name to its correct site_id
site_name_to_site_id = {}

# Iterate over unique site names in df
for site_name in df['site_name'].unique():
    # Get all site_ids for this site_name in df
    site_ids_in_df = df[df['site_name'] == site_name]['site_id'].unique()
    
    # Check if any of these site_ids exist in A
    matching_site_ids = A[A['site_id'].isin(site_ids_in_df)]['site_id'].unique()
    
    # If a match is found, map the site_name to the first matching site_id
    if len(matching_site_ids) > 0:
        site_name_to_site_id[site_name] = matching_site_ids[0]
    else:
        # Otherwise, keep the original site_id from df
        site_name_to_site_id[site_name] = site_ids_in_df[0]

# Update the site_id in df based on the mapping
df['site_id'] = df['site_name'].map(site_name_to_site_id)


In [425]:

# Counter IDs with more than one unique name
counter_id_name_pairs = df.groupby('counter_id')['counter_name'].nunique()
counter_id_name_filtered = counter_id_name_pairs[counter_id_name_pairs > 1]
counter_id_name_df = df[df['counter_id'].isin(counter_id_name_filtered.index)][['counter_id', 'counter_name']].drop_duplicates()

print("Counter IDs with more than one unique name:")
print(counter_id_name_df.sort_values(by='counter_id').to_string(index=False))

# Counter Names with more than one unique ID
counter_name_id_pairs = df.groupby('counter_name')['counter_id'].nunique()
counter_name_id_filtered = counter_name_id_pairs[counter_name_id_pairs > 1]
counter_name_id_df = df[df['counter_name'].isin(counter_name_id_filtered.index)][['counter_name', 'counter_id']].drop_duplicates()

print("\nCounter Names with more than one unique ID:")
print(counter_name_id_df.sort_values(by='counter_name').to_string(index=False))

# Site IDs with more than one unique name
site_id_name_pairs = df.groupby('site_id')['site_name'].nunique()
site_id_name_filtered = site_id_name_pairs[site_id_name_pairs > 1]
site_id_name_df = df[df['site_id'].isin(site_id_name_filtered.index)][['site_id', 'site_name']].drop_duplicates()

print("\nSite IDs with more than one unique name:")
print(site_id_name_df.sort_values(by='site_id').to_string(index=False))

# Site Names with more than one unique ID
site_name_id_pairs = df.groupby('site_name')['site_id'].nunique()
site_name_id_filtered = site_name_id_pairs[site_name_id_pairs > 1]
site_name_id_df = df[df['site_name'].isin(site_name_id_filtered.index)][['site_name', 'site_id']].drop_duplicates()

print("\nSite Names with more than one unique ID:")
print(site_name_id_df.sort_values(by='site_name').to_string(index=False))


Counter IDs with more than one unique name:
Empty DataFrame
Columns: [counter_id, counter_name]
Index: []

Counter Names with more than one unique ID:
Empty DataFrame
Columns: [counter_name, counter_id]
Index: []

Site IDs with more than one unique name:
Empty DataFrame
Columns: [site_id, site_name]
Index: []

Site Names with more than one unique ID:
Empty DataFrame
Columns: [site_name, site_id]
Index: []


In [426]:
df.nunique()


counter_id         83
counter_name       83
site_id            70
site_name          70
date            17543
latitude           84
longitude          85
result           1066
dtype: int64

In [ ]:
df = pd.concat([df, A], ignore_index=True)
duplicate_rows = combined_df[combined_df.duplicated()]
df = df.drop_duplicates()
# Check for NA values in the combined DataFrame
na_summary = df.isna().sum()
na_summary


In [427]:
def _encode_dates(X):
    X = X.copy()  # Modify a copy of X
    
    # Ensure 'date' is in datetime format
    X["date"] = pd.to_datetime(X["date"])
    
    # Extract date components
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Identify weekends (Saturday = 5, Sunday = 6)
    X["is_weekend"] = X["weekday"].isin([5, 6])
    
    # Get French holidays for all years in the dataset
    years = X["year"].unique()
    fr_holidays = holidays.France(years=years)
    
    # Identify holidays
    X["is_holiday"] = X["date"].dt.date.isin(fr_holidays)
    
    # Drop the original 'date' column
    return X


In [254]:
date_encoder = FunctionTransformer(_encode_dates, validate=False)
df = date_encoder.fit_transform(df)
df.head(2)

,counter_id,counter_name,site_id,date,latitude,longitude,year,month,day,weekday,hour,is_weekend,is_holiday
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,2020-01-01 01:00:00,48.83511,2.33338,2020,1,1,2,1,False,True
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,2020-01-01 03:00:00,48.83511,2.33338,2020,1,1,2,3,False,True


In [255]:
df = df.sort_values(["date", "counter_name"])

In [263]:
df.head(2)

,counter_id,counter_name,site_id,date,latitude,longitude,year,month,day,weekday,hour,is_weekend,is_holiday
132639,100044494-SC,10 avenue de la Grande Armée SE-NO,100044494,2020-01-01,48.874716,2.292439,2020,1,1,2,0,False,True
368690,100049408-SC,10 boulevard Auguste Blanqui NE-SO,100049408,2020-01-01,48.830680,2.353480,2020,1,1,2,0,False,True


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2247259 entries, 132639 to 2115858
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   counter_id    object        
 1   counter_name  object        
 2   site_id       int64         
 3   date          datetime64[ns]
 4   latitude      float64       
 5   longitude     float64       
 6   year          int32         
 7   month         int32         
 8   day           int32         
 9   weekday       int32         
 10  hour          int32         
 11  is_weekend    bool          
 12  is_holiday    bool          
dtypes: bool(2), datetime64[ns](1), float64(2), int32(5), int64(1), object(2)
memory usage: 167.2+ MB


counter_id      0
counter_name    0
site_id         0
site_name       0
date            0
latitude        0
longitude       0
result          0
dtype: int64

In [ ]:
y = df['result']
df.drop(columns=['result'], inplace=True)

In [276]:
def train_test_split_temporal(X, y, delta_threshold="30 days"):
    
    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

In [ ]:
X = df.copy()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2559980 entries, 0 to 2744085
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   counter_id    object        
 1   counter_name  object        
 2   site_id       int64         
 3   site_name     object        
 4   date          datetime64[ns]
 5   latitude      float64       
 6   longitude     float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 156.2+ MB


In [183]:

# One-hot encode the categorical variables
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns
categorical_cols = categorical_cols.append(pd.Index(['latitude', 'longitude']))
onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
categorical_encoded = onehot_encoder.fit_transform(X[categorical_cols])

# Convert the encoded array to a DataFrame
categorical_encoded_df = pd.DataFrame(
    categorical_encoded, 
    columns=onehot_encoder.get_feature_names_out(categorical_cols),
    index=X.index
)

# Drop original categorical columns and add the encoded columns
X = X.drop(columns=categorical_cols)
X = pd.concat([X, categorical_encoded_df], axis=1)



In [184]:
X.head()

,date,year,month,day,weekday,hour,is_weekend_False,is_weekend_True,is_holiday_False,is_holiday_True,...,longitude_0.8285437027795091,longitude_0.8310112982602078,longitude_0.8339225064115894,longitude_0.8426561308657377,longitude_0.8521799403895471,longitude_0.8556179385873719,longitude_0.9366523706886589,longitude_0.9366534969155076,longitude_0.9974353642475933,longitude_1.0
132639,2020-01-01,0.0,0.0,0.0,0.333333,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
368690,2020-01-01,0.0,0.0,0.0,0.333333,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26233,2020-01-01,0.0,0.0,0.0,0.333333,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183403,2020-01-01,0.0,0.0,0.0,0.333333,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8808,2020-01-01,0.0,0.0,0.0,0.333333,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
# Numerical scaling
numerical_cols = X.select_dtypes(include=['int32', 'int64', 'float64']).columns
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(X[numerical_cols])

# Convert the scaled array to a DataFrame
numerical_scaled_df = pd.DataFrame(
    numerical_scaled, 
    columns=numerical_cols,
    index=X.index
)

# Drop original numerical columns and add scaled columns
X = X.drop(columns=numerical_cols)
X = pd.concat([X, numerical_scaled_df], axis=1)


# # Combine all features efficiently using NumPy
# X_combined = np.hstack([X[categorical_cols], X[numerical_cols]])

# # Add the date column back for temporal split
# X_combined_df = pd.DataFrame(X_combined, columns=[f"feature_{i}" for i in range(X_combined.shape[1])])
# X_combined_df["date"] = pd.to_datetime(X["date"])  # Ensure date is datetime

# Step 3: Temporal Train-Test Split



In [186]:
X.head

<bound method NDFrame.head of                        date  year  month  day   weekday  hour  \
132639  2020-01-01 00:00:00   0.0    0.0  0.0  0.333333   0.0   
368690  2020-01-01 00:00:00   0.0    0.0  0.0  0.333333   0.0   
26233   2020-01-01 00:00:00   0.0    0.0  0.0  0.333333   0.0   
183403  2020-01-01 00:00:00   0.0    0.0  0.0  0.333333   0.0   
8808    2020-01-01 00:00:00   0.0    0.0  0.0  0.333333   0.0   
...                     ...   ...    ...  ...       ...   ...   
1196058 2021-12-31 23:00:00   1.0    1.0  1.0  0.666667   1.0   
1677858 2021-12-31 23:00:00   1.0    1.0  1.0  0.666667   1.0   
2063298 2021-12-31 23:00:00   1.0    1.0  1.0  0.666667   1.0   
2098338 2021-12-31 23:00:00   1.0    1.0  1.0  0.666667   1.0   
2115858 2021-12-31 23:00:00   1.0    1.0  1.0  0.666667   1.0   

         is_weekend_False  is_weekend_True  is_holiday_False  is_holiday_True  \
132639                1.0              0.0               0.0              1.0   
368690                1.0  

In [187]:
X_train_split, y_train_split, X_test_split, y_test_split = train_test_split_temporal(X, y)

# # Step 4: Drop datetime columns after splitting
# for df in [X_train_split, X_test_split]:
#     datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns
#     df = df.drop(columns=datetime_columns)

for df_name in ["X_train_split", "X_test_split"]:
    df = locals()[df_name]
    datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns
    if not datetime_columns.empty:
        print(f"Dropping datetime columns: {datetime_columns.tolist()} from {df_name}")
        locals()[df_name] = df.drop(columns=datetime_columns)
# Convert to float in one step
X_train_split = X_train_split.astype(float)
X_test_split = X_test_split.astype(float)

Dropping datetime columns: ['date'] from X_train_split
Dropping datetime columns: ['date'] from X_test_split


In [188]:
import utils

final_test = utils.get_test_data()

final_test.head(2)


,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
17081,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2021-09-10 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233
18655,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2021-09-10 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233


In [189]:
final_test.drop(columns=['coordinates','counter_technical_id'], inplace=True)

In [190]:
final_test.drop(columns=['site_id', 'site_name', 'counter_name', 'counter_id', 'counter_installation_date'], inplace=True)

In [191]:
final_test.head(2)

,date,latitude,longitude
17081,2021-09-10 01:00:00,48.840801,2.333233
18655,2021-09-10 01:00:00,48.840801,2.333233


In [192]:
date_encoder = FunctionTransformer(_encode_dates, validate=False)
final_test = date_encoder.fit_transform(final_test)
final_test.head(2)

,date,latitude,longitude,year,month,day,weekday,hour,is_weekend,is_holiday
17081,2021-09-10 01:00:00,48.840801,2.333233,2021,9,10,4,1,False,False
18655,2021-09-10 01:00:00,48.840801,2.333233,2021,9,10,4,1,False,False


In [193]:

# One-hot encode the categorical variables
test_categorical_cols = final_test.select_dtypes(include=['object', 'category', 'bool']).columns
test_categorical_cols = test_categorical_cols.append(pd.Index(['latitude', 'longitude']))
onehot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
test_categorical_encoded = onehot_encoder.fit_transform(final_test[test_categorical_cols])

# Convert the encoded array to a DataFrame
test_categorical_encoded_df = pd.DataFrame(
    test_categorical_encoded, 
    columns=onehot_encoder.get_feature_names_out(test_categorical_cols),
    index=final_test.index
)

# Drop original categorical columns and add the encoded columns
final_test = final_test.drop(columns=test_categorical_cols)
final_test = pd.concat([final_test, test_categorical_encoded_df], axis=1)



In [194]:
test_categorical_encoded_df.head(2)

,is_weekend_False,is_weekend_True,is_holiday_False,latitude_48.82636,latitude_48.829523,latitude_48.830331,latitude_48.83421,latitude_48.83436,latitude_48.83848,latitude_48.83977,...,longitude_2.368852,longitude_2.37376,longitude_2.375429,longitude_2.37587,longitude_2.377,longitude_2.38482,longitude_2.38531,longitude_2.38699,longitude_2.400551,longitude_2.40969
17081,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18655,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [195]:
# Numerical scaling
test_numerical_cols = final_test.select_dtypes(include=['int32', 'int64', 'float64']).columns
scaler = MinMaxScaler()
test_numerical_scaled = scaler.fit_transform(final_test[test_numerical_cols])

# Convert the scaled array to a DataFrame
test_numerical_scaled_df = pd.DataFrame(
    test_numerical_scaled, 
    columns=test_numerical_cols,
    index=final_test.index
)

# Drop original numerical columns and add scaled columns
final_test = final_test.drop(columns=test_numerical_cols)
final_test = pd.concat([final_test, test_numerical_scaled_df], axis=1)

In [196]:

datatype_counts = final_test.dtypes.value_counts()
datatype_counts


float64           68
datetime64[us]     1
Name: count, dtype: int64

In [197]:

test_datetime_columns = final_test.select_dtypes(include=['datetime64[ns]']).columns
if not test_datetime_columns.empty:
        print(f"Dropping datetime columns: {test_datetime_columns.tolist()} from {final_test}")
        final_test = final_test.drop(columns=test_datetime_columns)

# Convert to float in one step
final_test = final_test.astype(float)

Dropping datetime columns: ['date'] from                      date  year  month       day   weekday      hour  \
17081 2021-09-10 01:00:00   0.0    0.0  0.310345  0.666667  0.043478   
18655 2021-09-10 01:00:00   0.0    0.0  0.310345  0.666667  0.043478   
3124  2021-09-10 01:00:00   0.0    0.0  0.310345  0.666667  0.043478   
4147  2021-09-10 01:00:00   0.0    0.0  0.310345  0.666667  0.043478   
48210 2021-09-10 01:00:00   0.0    0.0  0.310345  0.666667  0.043478   
...                   ...   ...    ...       ...       ...       ...   
42131 2021-10-18 21:00:00   0.0    1.0  0.586207  0.000000  0.913043   
43042 2021-10-18 21:00:00   0.0    1.0  0.586207  0.000000  0.913043   
43929 2021-10-18 21:00:00   0.0    1.0  0.586207  0.000000  0.913043   
5707  2021-10-18 21:00:00   0.0    1.0  0.586207  0.000000  0.913043   
6855  2021-10-18 21:00:00   0.0    1.0  0.586207  0.000000  0.913043   

       is_weekend_False  is_weekend_True  is_holiday_False  latitude_48.82636  \
17081        

In [198]:
models = {
    "XGBoost": xgb.XGBRegressor(random_state=42, verbosity=1),
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    # "Random Forest": RandomForestRegressor(
    #     n_estimators=100,  # Fewer trees
    #     max_depth=20,     # Limit depth
    #     min_samples_split=5,
    #     min_samples_leaf=2,
    #     random_state=42,
    #     n_jobs=-1         # Utilize multiple cores
    # ),
}

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train_split, y_train_split)

    # Predict on the test set
    y_pred = model.predict(X_test_split)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test_split, y_pred))
    results[name] = rmse
    print(f"Model: {name}, RMSE: {rmse}")

# Convert results to a DataFrame and display
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['RMSE']).sort_values(by='RMSE')

# Display the results
print(results_df)



Model: XGBoost, RMSE: 2.185407371337057
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 392
[LightGBM] [Info] Number of data points in the train set: 1781299, number of used features: 163
[LightGBM] [Info] Start training from score 2.346541
Model: LightGBM, RMSE: 2.267581642990328
              RMSE
XGBoost   2.185407
LightGBM  2.267582


In [ ]:
for name, model in models.items():
    predictions = model.predict(final_test)
    submission = pd.DataFrame({"id": final_test.index, "log_bike_count": predictions.flatten()})
    submission_path = f"submission_{name}.csv"
    submission.to_csv(submission_path, index=False)

In [103]:
X_train_split.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1781299 entries, 132639 to 2115138
Columns: 466 entries, is_weekend to counter_installation_date_21-07-21
dtypes: float64(466)
memory usage: 6.2 GB
